In [1]:
%load_ext autoreload
%autoreload 2
import gc
import numpy as np
import pandas as pd
import sys
import re
from glob import glob
import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
#  sys.path.append(f"/mnt/c/Git/go/kaggle/github/library/")
import utils
from utils import logger_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()
    
from tqdm import tqdm

#========================================================================
# Global Variable
sys.path.append(f"../py")
from info_home_credit import hcdr_key_cols
key, target, ignore_list = hcdr_key_cols()
#========================================================================

2019-03-20 17:26:31,098 utils 346 [INFO]    [logger_func] start 


In [4]:
#========================================================================
# Create New Dataset
new_dataset = ""
bq_cmd = f"bq --location=US mk {new_dataset}"
bq_client.create_dataset(dataset='test2017')

# Delete Datset
bq_cmd = f"bq --location=US rm {new_dataset}"
bq_client.delete_dataset(dataset='test2017')

# Delete Table
table = ""
bq_cmd = f"bq --location=US rm {new_dataset}.{table}"
bq_client.delete_table('hori.tmp')

# Save to GCloud Strage
gst =  "gs://input_hori/"
export_fname = ""
ds_table = "hori.installment"
bq_cmd = f"bq extract --destination_format CSV {ds_table} {gst}{export_fname}"
#========================================================================


#========================================================================
# Create New Table from Local Dataset

import os
option = "--autodetect"
dataset = 'hori'
new_table = 'credit'
data_path = '../input/credit_card_balance.csv'
# os.system("bq --location=US mk dataset")
bq_cmd = f"bq load {option} {dataset}.{new_table} {data_path}"
os.system(bq_cmd)
#========================================================================

0

In [45]:
#========================================================================
# Create New Table from Query

query = """
select * 
from `hori.installment`
limit 100
;
"""
project = 'horikoshi-ml-224313'
dataset = 'hori'
new_table = 'tmp2'
bq_cmd = f"bq --location=US query --destination_table {project}:{dataset}.{new_table} --use_legacy_sql=false '{query}'"
os.system(bq_cmd)
#========================================================================

0

In [30]:
import joblib
from google.cloud import bigquery

def bq_load(query):
    bq_client = bigquery.Client.from_service_account_json(f'{HOME}/privacy/horikoshi-ml-a13ef6f2f937.json')
    df = bq_client.query(query).to_dataframe()
    return df

query = """
select * 
from `hori.tmp`
"""
df = bq_load(query)
df

exec_date = '20190301'
queries = {}
query_file_list = glob.glob("./sql/{exec_date}*.sql")
for query_file in query_list:
    with open(query_file) as f:
        query = f.read()
    feature_name = utils.get_filename(query, delimiter='sql').replace(f'{exec_date}_', '')
    queries[feature_name] = query

r = joblib.Parallel(n_jobs=32)(
    joblib.delayed(bq_load)(query, feature_name, exec_date)
    for feature_name, query in queries.items()
)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,2594498,332851,4.0,102,-374.0,-289.0,801.720,801.720
1,1057358,272685,4.0,103,-1103.0,-1053.0,1350.000,1350.000
2,1398473,332164,4.0,103,-316.0,-316.0,56.700,56.700
3,2214469,285405,5.0,28,-1714.0,-1721.0,36195.435,36195.435
4,1980816,309057,5.0,27,-41.0,-41.0,34861.680,34861.680
5,2320248,171743,6.0,26,-137.0,-165.0,21803.355,21803.355
6,2476896,136375,6.0,26,-228.0,-239.0,33804.180,33804.180
7,1803195,182943,7.0,32,-75.0,-85.0,17394.300,17394.300
8,2425630,269987,7.0,32,-131.0,-136.0,13201.245,13201.245
9,1289873,122475,9.0,32,-192.0,-229.0,26915.625,26915.625
